In [250]:
import csv 
import re
import pandas as pd
import fitz
import os

# TO DO LIST for clean (each individual)
- Page splits
- Same prof, different locations (multiple new lines)

In [251]:
def extract_text_from_pdf(pdf_path):
    # Open the provided PDF file
    doc = fitz.open(pdf_path)
    text = ""
    
    # Extract text from each page
    for page in doc:
        text += page.get_text()
    
    doc.close()
    return text

In [252]:
# class replacement lines
name_list = [
    'COMPUTER\nSCIENCE\nFOR\nSTEM',
    'INTRODUCTION\nTO\nCOMPUTER\nSCIENCE',
    'DATA\nSTRUCTURES\nAND\nALGORITHMS',
    'FOUNDATIONAL\nPROGRAMMING\nCONCEPTS &\nDESIGN'
    'DATA\nSTRUCTURES\nAND\nALGORITHMS', 
    'PYTHON-BASED\nCOMPUTING:\nBUILDING A\nSENSOR SYSTEM', 
    'INTRODUCTION\nTO THE LINUX\nOPERATING\nSYSTEM', 
    'SOFTWARE\nENGINEERING', 
    'COMPUTER\nORGANIZATION', 
    'DISCRETE\nMATHEMATICS',
    'ADVANCED\nSOFTWARE\nENGINEERING',
    'PRINCIPLES OF\nPROGRAMMING\nLANGUAGES', 
    'DATA BASE\nMANAGEMENT', 
    'ALGORITHMS', 
    'GAME THEORY\nAND NETWORKS',
    'INFORMATION\nSECURITY AND\nPRIVACY', 
    'GRADUATE\nTHESIS /\nDISSERTATION\nRESEARCH\nCREDIT',
    'INTRODUCTION\nTO\nBIOINFORMATICS',
    'COMPOTER\nSCIENCE\nHONORS THESIS',
    'MASTERS\nPROJECT\nCREDITS',
    'HUMAN-ROBOT\nINTERACTION',
    'OPERATING\nSYSTEMS'

    'INTRODUCTION\nTO COMPUTER\nSCIENCE - LAB',
    'INTRODUCTION\nTO COMPUTER\nSCIENCE',
    'PROGRAMMING\nCONCEPTS', 
    'DATA\nSTRUCTURES',
]

# Extract text from PDFs

In [253]:
# find all files with .pdf extension in directory schedules
pdf_files = [f for f in os.listdir('../schedules') if f.endswith('.pdf')]

In [254]:
# extract text from pdf files
for pdf_path in pdf_files: 
    text = extract_text_from_pdf("../schedules/" + pdf_path)
    # save text
    with open("../schedules/raw_" + pdf_path.replace(".pdf", "") + '.txt', 'w') as f:

        # strip extra spaces from end of lines
        text = re.sub(r'\s+$', '', text, flags=re.MULTILINE)

        # remove header details
        header_end_marker = 'Location Attribute'
        end_column_location = text.index(header_end_marker)
        text = text[end_column_location + len(header_end_marker) + 1:]

        # renmove footer details
        footer_marker = 'New Search\n© 2024 Ellucian Company L.P. and its affiliates.'
        text = text.replace(footer_marker, '')  

        # replace UG schld with UG_Req 
        ug_req_schld_flag = 'UG\nRequired\nCrse for\nSchdlng'
        ta_ra_flag = 'Summer\nGraduate\nTA/RA Fee'

        text = text.replace(ug_req_schld_flag, 'UG_Req\n').replace(ta_ra_flag, 'Summer_Grad_TARA_Fee\n')

        # replace ONLINE NR tag
        text = text.replace('ONLINE NR', 'ONLINE')
        text = text.replace('ONLINE UG_Req\n', 'ONLINE\nUG_Req\n')
        for code in ["NR", "C"]:
            text = text.replace(f"ONLINE\n{code}", f"ONLINE\n\n{code}")
            text = text.replace(f"UG_Req\n{code}", f"UG_Req\n\n{code}")

        # replace TBA with false scheduling information
        text = text.replace('\nTBA','\nAAA\n00:00\nzm-\n00:00\nzm')

        # replace class names
        for name in name_list:
            text = text.replace(name, name.replace('\n', ' '))

        f.write(text)

In [294]:
# open file to read 
with open("../schedules/clean_Fall21.txt", 'r') as f:
    text = f.read()

In [295]:
# replace block endings
pattern = r'([A-Z]{2} \d{3})\n(NR|C)\n(\d{5})'
nr_pattern = r'(NR NR)\n(NR|C)\n(\d{5})'
replacement = r'\1\n\n\2\n\3'

text = re.sub(pattern, replacement, text)
text = re.sub(nr_pattern, replacement, text)

In [296]:
block_list = text.split('\n\n')
print(block_list)

['NR\n80291 CSCI\n101\nA\nC\n3.000\nINTRODUCTION TO COMPUTER SCIENCE\nMWF\n01:00\npm-\n01:50\npm\n61\n60\n1\n0\n0\n0\n0\n0\n0\nMackenzie\nGayle Hull\n(P), Tracy K\nCamp ,\nMegan\nKinsley\nShapiro\n08/23-\n12/17\nMZ 108\nUG_Req', 'C\n80268 CSCI\n101\nB\nC\n3.000\nINTRODUCTION TO COMPUTER SCIENCE\nMWF\n02:00\npm-\n02:50\npm\n63\n63\n0\n0\n0\n0\n0\n0\n0\nRobert Haven\nThompson\n(P), Tracy K\nCamp ,\nMegan\nKinsley\nShapiro\n08/23-\n12/17\nAH 330\nUG_Req', 'NR\n80302 CSCI\n101\nC\nC\n3.000\nINTRODUCTION TO COMPUTER SCIENCE\nMWF\n03:00\npm-\n03:50\npm\n64\n60\n4\n0\n0\n0\n0\n0\n0\nRobert Haven\nThompson\n(P), Tracy K\nCamp ,\nMegan\nKinsley\nShapiro\n08/23-\n12/17\nBE 243\nUG_Req', 'NR\n80811 CSCI\n101\nD\nC\n3.000\nINTRODUCTION TO COMPUTER SCIENCE\nMWF\n01:00\npm-\n01:50\npm\n63\n62\n1\n0\n0\n0\n0\n0\n0\nArmando Eric\nOcampo (P),\nTracy K\nCamp ,\nMegan\nKinsley\nShapiro\n08/23-\n12/17\nBB W210\nUG_Req', 'C\n80960 CSCI\n101\nE\nC\n3.000\nINTRODUCTION TO COMPUTER SCIENCE\nMWF\n02:00\npm-\n0

In [297]:
split_list = []
for index,block in enumerate(block_list): 
    container = []
    for line in block.split('\n'):
        if line != '':
            container.append(line)
    print(f"LINE #{index}")
    if container != []:
        try: 
            # add end flag (for counts)
            if container[-1] != 'UG_Req' and container[-1] != 'Summer_Grad_TARA_Fee':
                container.append('No_Special_Notes')

            # remove start flag (for counts)
            if container[0] != 'C' and container[0] != 'NR': 
                raise KeyError
            
            # combine credit ranges
            if container[5][-1] == '-':
                container[5] = container[5] + container[6]
                container.remove(container[6])

            # combine dates
            container[-4] = container[-4] + container[-3]
            container.remove(container[-3])

            # combine times
            container[8] = container[8] + container[9] + container[10] + container[11]
            for i in range(11,8,-1):
                container.remove(container[i])

            # combine instructors
            container[18] = " ".join([container[i].replace(" ,", ",") for i in range(18, len(container) - 3)])
            for i in range(len(container) - 4, 18, -1):
                container.remove(container[i])

            # split CRN from Crse #
            l = container[1].split(" ") 
            container[1] = l[0]
            container.insert(2, l[1])

            # add to container
            split_list.append(container)

            print("\t", container)
            print("\n")

        except: 
            print("ERROR")
            print(block)
            print(container)

print(split_list)

LINE #0
	 ['NR', '80291', 'CSCI', '101', 'A', 'C', '3.000', 'INTRODUCTION TO COMPUTER SCIENCE', 'MWF', '01:00pm-01:50pm', '61', '60', '1', '0', '0', '0', '0', '0', '0', 'Mackenzie Gayle Hull (P), Tracy K Camp, Megan Kinsley Shapiro', '08/23-12/17', 'MZ 108', 'UG_Req']


LINE #1
	 ['C', '80268', 'CSCI', '101', 'B', 'C', '3.000', 'INTRODUCTION TO COMPUTER SCIENCE', 'MWF', '02:00pm-02:50pm', '63', '63', '0', '0', '0', '0', '0', '0', '0', 'Robert Haven Thompson (P), Tracy K Camp, Megan Kinsley Shapiro', '08/23-12/17', 'AH 330', 'UG_Req']


LINE #2
	 ['NR', '80302', 'CSCI', '101', 'C', 'C', '3.000', 'INTRODUCTION TO COMPUTER SCIENCE', 'MWF', '03:00pm-03:50pm', '64', '60', '4', '0', '0', '0', '0', '0', '0', 'Robert Haven Thompson (P), Tracy K Camp, Megan Kinsley Shapiro', '08/23-12/17', 'BE 243', 'UG_Req']


LINE #3
	 ['NR', '80811', 'CSCI', '101', 'D', 'C', '3.000', 'INTRODUCTION TO COMPUTER SCIENCE', 'MWF', '01:00pm-01:50pm', '63', '62', '1', '0', '0', '0', '0', '0', '0', 'Armando Eric Oca

In [298]:
# save to csv
with open('../schedules/Fall21.csv', 'w') as f:
    writer = csv.writer(f)

    # write header
    writer.writerow(['Select', 'CRN', 'Subj', 'Section', 'CMP', 
                        'Credits', 'Title', 'Days', 'Time', 
                        'Cap', 'Act', 'WL Cap', 'WL Act', 'WL Rem', 'XL Cap', 'XL Act', 'XL Rem', 
                        'Instructors', 'Date', 'Location', 'Attributes'])
    writer.writerows(split_list)